In [17]:
import sys
import requests
import pandas as pd 
import math

sell_SL_diff_7d = []
sell_SL_diff_1d = []
buy_SL_diff_7d = []
buy_SL_diff_1d = []
sell_sold_5pct_1d = []
profit_est = []

url_items_tp_data = (   'https://api.datawars2.ie/gw2/v1/items/json?fields='
                        'id,name,buy_price,sell_price,roi,profit,'
                        '7d_buy_listed,7d_buy_sold,7d_sell_listed,7d_sell_sold,'
                        '1d_buy_listed,1d_buy_sold,1d_sell_listed,1d_sell_sold')


result = requests.get(url_items_tp_data)

if(result.status_code == 200):
    data = result.json()
    df = pd.json_normalize(data)

else:
	sys.exit("Status code not 200")


for i, row in df.iterrows():

    sell_SL_diff_7d.append(row['7d_sell_sold'] - row['7d_sell_listed'])
    sell_SL_diff_1d.append(row['1d_sell_sold'] - row['1d_sell_listed'])
    buy_SL_diff_7d.append(row['7d_buy_sold'] - row['7d_buy_listed'])
    buy_SL_diff_1d.append(row['1d_buy_sold'] - row['1d_buy_listed'])

    if(not math.isnan(row['1d_sell_sold'])):
        sell_sold_5pct_1d.append(int(row['1d_sell_sold'] * 0.05))
        profit_est.append(row['profit'] * sell_sold_5pct_1d[-1])

    else:
        sell_sold_5pct_1d.append(float('nan'))
        profit_est.append(float('nan'))

df['sell_SL_diff_7d'] = sell_SL_diff_7d
df['sell_SL_diff_1d'] = sell_SL_diff_1d
df['buy_SL_diff_7d'] = buy_SL_diff_7d
df['buy_SL_diff_1d'] = buy_SL_diff_1d

df['ss_speed'] = sell_sold_5pct_1d
df['profit_est'] = profit_est

df['roi'] = df['roi'] * 100

# df.round()
# df.round({"roi":2})

df = df.round(decimals=0).astype(object)
df = df.convert_dtypes()

# df.dtypes

id                   Int64
1d_buy_listed        Int64
1d_buy_sold          Int64
1d_sell_listed       Int64
1d_sell_sold         Int64
7d_buy_listed        Int64
7d_buy_sold          Int64
7d_sell_listed       Int64
7d_sell_sold         Int64
buy_price            Int64
name                string
profit               Int64
roi                Float64
sell_price           Int64
sell_SL_diff_7d      Int64
sell_SL_diff_1d      Int64
buy_SL_diff_7d       Int64
buy_SL_diff_1d       Int64
ss_speed             Int64
profit_est           Int64
dtype: object

In [18]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
	display(df[[	
		'name',
		'sell_price',
		'buy_price',
		'profit',
		'profit_est',
		'ss_speed',
		'roi',


		# '7d_buy_listed',
		'7d_buy_sold',
		'buy_SL_diff_7d',

		# '1d_buy_listed',
		'1d_buy_sold',
		'buy_SL_diff_1d',

		# '7d_sell_listed',
		'7d_sell_sold',
		'sell_SL_diff_7d',

		# '1d_sell_listed',
		'1d_sell_sold',
		'sell_SL_diff_1d'

		]]

	.loc	[
		(df['profit'] > 5) &
		(df['roi'] > 1) &
		(df['7d_sell_sold'] > 110) &
		(df['1d_sell_sold'] > 20) &
		# (df['7d_buy_sold'] > 500) &
		(df['sell_SL_diff_7d'] > 0) &
		(df['sell_SL_diff_1d'] > -10) #&

		# (df['buy_SL_diff_7d'] > 0) &
		# (df['buy_SL_diff_1d'] > -10) #&
		]

	.sort_values(by='profit_est', ascending=False)
	# .sort_values(by='ss_speed', ascending=False)

	# .head(30)

	.style
		.hide_index()
		.bar(subset=[	'buy_SL_diff_7d',
				'buy_SL_diff_1d',
				'sell_SL_diff_7d',
				'sell_SL_diff_1d'
				], align='zero', color=['#d65f5f', '#5fba7d'])
		.set_properties(**{'background-color': '#8ba6a0'}, subset=['7d_buy_sold', 'buy_SL_diff_7d'])
		.set_properties(**{'background-color': '#7b838f'}, subset=['1d_buy_sold', 'buy_SL_diff_1d'])
		.set_properties(**{'background-color': '#8ba6a0'}, subset=['7d_sell_sold', 'sell_SL_diff_7d'])
		.set_properties(**{'background-color': '#7b838f'}, subset=['1d_sell_sold', 'sell_SL_diff_1d'])
		.set_properties(**{'background-color': '#b8a73b'}, subset=['profit_est'])
	# .shape[0]
	)

name,sell_price,buy_price,profit,profit_est,ss_speed,roi,7d_buy_sold,buy_SL_diff_7d,1d_buy_sold,buy_SL_diff_1d,7d_sell_sold,sell_SL_diff_7d,1d_sell_sold,sell_SL_diff_1d
Superior Sigil of Strength,44943,34627,3575,39325,11,10.320000,460,-426,63,-80,1569,148,229,17
Superior Sigil of Transference,43592,34200,2853,28530,10,8.340000,625,-3505,101,-172,1601,251,206,4
+2 Agony Infusion,396,170,167,22545,135,98.000000,9619,-4482,5787,-4370,8722,2605,2707,1585
Superior Rune of the Thief,25575,20024,1715,20580,12,8.560000,926,-3176,71,-149,2677,231,255,3
Ogre Sharpening Stone,3679,1444,1683,20196,12,116.560000,426,30,186,156,2146,324,249,89
Major Rune of Balthazar,721,195,418,20064,48,214.280000,695,395,0,0,1499,451,961,812
Berserker's Emblazoned Coat,107492,72500,18868,18868,1,26.020000,23,-130,3,-25,145,2,21,-5
Superior Sigil of Doom,40738,29961,4666,18664,4,15.570000,644,-1649,32,-429,811,70,82,-6
Cleric's Hard Inscription,1699,832,612,15300,25,73.570000,384,-4255,0,-534,1212,678,503,451
Superior Rune of the Herald,26761,7448,15299,15299,1,205.400000,189,-945,63,-407,333,67,38,6
